# Função

In [11]:
#-----------------------------------------------------------------------------------
#        Obtêm as informações do radar a partir do dado bruto (HDF5)
#-----------------------------------------------------------------------------------
def get_info_radar(radar):

    """
    Obtêm as informações dos dados do radar

    Parâmetros de entrada:
                           radar (object): objeto do pyart

    Parâmetros de saída:
                         dic (dicionário): informações do radar, como:
                           date_start(list): ano, mês, dia hora, minuto inicial do scan do radar
                           date_end(list): ano, mês, dia hora, minuto final do scan do radar
                           lon_radar(float): longitude do radar em graus
                           lat_radar(float): latitude do radar em graus
                           alt_radar(float): altitude do radar em metros
                           radar_frequency(float): frequência do radar em Hz
                           radar_beam_width(float): largura do feixe em graus
                           nsweeps(int): quantidade de elevações
                           sweeps(list): ângulos de elevação em graus
                           nrays(list): quantidade de azimutes para cada elevação
                           ngates(int): quantidade de bins
                           radial_resolution(float): resolução radial do radar em metros
                           ray_angle_res(float): resolução azimutal em graus
                           lonmin(float): longitude mínima da matriz de dados
                           lonmax(float): longitude máxima da matriz de dados
                           latmin(float): latitude mínima da matriz de dados
                           latmax(float): latitude máxima da matriz de dados
                           variables(list): variáveis disponíveis

    Exemplo:
             dic = get_info_radar(radar)
    """

    # importa biblioteca de datas
    from netCDF4 import num2date

    # campos disponíveis
    variaveis = radar.fields.keys()

    # latitude e longitude do radar
    lon_radar, lat_radar, alt_radar = radar.longitude['data'][0], radar.latitude['data'][0], radar.altitude['data'][0]

    # informações sobre o tempo
    data_ini = num2date(radar.time['data'][0], radar.time['units'] )
    data_end = num2date(radar.time['data'][-1], radar.time['units'] )

    # extrai o ano, mes dia, hora e minuto inicial da imagem
    anoi, mesi, diai, hori, mini, segi = str(data_ini.year), str(data_ini.month).zfill(2), str(data_ini.day).zfill(2), str(data_ini.hour).zfill(2), str(data_ini.minute).zfill(2), str(data_ini.second).zfill(2)
    anof, mesf, diaf, horf, minf, segf = str(data_end.year), str(data_end.month).zfill(2), str(data_end.day).zfill(2), str(data_end.hour).zfill(2), str(data_end.minute).zfill(2), str(data_end.second).zfill(2)

    #--------------------------------------------------------------#
    #                  FORÇANDO A FREQUÊNCIA
    #--------------------------------------------------------------#
    # No dado não esta vindo o valor da frequência do radar banda-S. 
    # Assim definimos a frequência manualmente
    radar.instrument_parameters = {'frequency': {'data': [2.8e9]}}
    
    # frequência do radar
    radar_frequency = radar.instrument_parameters['frequency']['data'][0]

    # largura do feixe
    radar_beam_width = 0.96 # Não tem o valor da largura do feixe nos dado: radar_beam_width = radar.instrument_parameters['radar_beam_width_h']['data'][0]

    # quantidade TOTAL de bins do feixe
    ngates = radar.ngates

    # quantidade de elevações
    nsweeps = radar.nsweeps

    # quantidade de azimutes por elevação
    total_azimuths = radar.azimuth['data'].shape[0]
    index_azimuths = radar.sweep_start_ray_index['data'].tolist()
    index_azimuths.append(total_azimuths)
    qte_azim = [index_azimuths[i+1] - index_azimuths[i] for i in range(nsweeps)]

    # resolução radial
    radial_resolution = radar.range['meters_between_gates']

    # informação apenas da resolução ângulo azimutal
    ray_angle_res = -999.9 # ray_angle_res = radar.ray_angle_res['data'][0]

    # ângulos de elevação
    sweeps =  radar.fixed_angle['data']

    # definindo a extensão dos dados em termos de latitude/longitude
    lats = radar.gate_latitude
    lons = radar.gate_longitude
    lonmin = lons['data'].min()
    lonmax = lons['data'].max()
    latmin = lats['data'].min()
    latmax = lats['data'].max()

    # coloca as variáveis num dicionário
    dic = {
            'date_start(year,month,day,hour,minute)': [anoi, mesi, diai, hori, mini],
            'date_end(year,month,day,hour,minute)': [anof, mesf, diaf, horf, minf],
            'lon_radar(degree)': lon_radar,
            'lat_radar(degree)': lat_radar,
            'alt_radar(meters)': alt_radar,
            'radar_frequency(Hz)': radar_frequency,
            'radar_beam_width(degree)': radar_beam_width,
            'nsweeps(#)': nsweeps,
            'sweeps(degree)': list(sweeps),
            'nrays(#)': qte_azim,
            'ngates(#)': ngates,
            'radial_resolution(meters)': radial_resolution,
            'ray_angle_res(degree)': ray_angle_res,
            'lonmin(degree)': lonmin,
            'lonmax(degree)': lonmax,
            'latmin(degree)': latmin,
            'latmax(degree)': latmax,
            'variables': list(variaveis)

           }

    return dic

# Informações do radar

In [12]:
# leitura do dado 
import pyart                              
filename = 'R13227490_202303231840.hdf'
radar = pyart.aux_io.read_odim_h5(f'input/2023-03-23/{filename}')

# processa a função que extrai as infomações do dado do radar
dic = get_info_radar(radar)

# salva arquivo
with open(f'output/02_INFO_RADAR/info_radar_LONTRAS.csv', 'w') as f:
    for key in dic.keys():
        f.write("%s= %s\n"%(key, dic[key]))

In [13]:
# mostra as informações do radar na tela
dic

{'date_start(year,month,day,hour,minute)': ['2023', '03', '23', '18', '40'],
 'date_end(year,month,day,hour,minute)': ['2023', '03', '23', '18', '45'],
 'lon_radar(degree)': -49.461029052734375,
 'lat_radar(degree)': -27.230154037475586,
 'alt_radar(meters)': 895.0,
 'radar_frequency(Hz)': 2800000000.0,
 'radar_beam_width(degree)': 0.96,
 'nsweeps(#)': 10,
 'sweeps(degree)': [0.5, 1.0, 2.3, 3.3, 4.2, 5.2, 6.2, 8.0, 10.0, 12.0],
 'nrays(#)': [348, 348, 348, 348, 348, 348, 392, 392, 461, 461],
 'ngates(#)': 1920,
 'radial_resolution(meters)': 125.0,
 'ray_angle_res(degree)': -999.9,
 'lonmin(degree)': -51.886562,
 'lonmax(degree)': -47.035484,
 'latmin(degree)': -29.38673,
 'latmax(degree)': -25.073557,
 'variables': ['total_power_horizontal',
  'velocity_horizontal',
  'velocity_vertical',
  'total_power_vertical',
  'reflectivity_horizontal',
  'reflectivity_vertical',
  'differential_reflectivity',
  'cross_correlation_ratio',
  'differential_phase',
  'specific_differential_phase']}